# Performance: Reflow & Repaint in JavaScript 

## Introduction 

This lecture explains how to measure JavaScript code performance and why understanding **reflow** and **repaint** in the browser is critical for writing efficient DOM‑manipulation code.  The instructor uses the example of creating and displaying 100 paragraphs in the UI using different coding strategies and then compares their performance.  The lecture also introduces **`performance.now()`** and **`DocumentFragment`** as tools for performance optimization. 

***

## Measuring Code Performance in JavaScript 

### Why measure performance? 

- To compare two implementations and decide which code is faster or slower.   
- To quantify performance rather than relying on intuition.   
- To identify and adopt better patterns for DOM updates so the UI feels smoother and more responsive. 

### Using `performance.now()` 

- `performance.now()` is a standard JavaScript API that returns a **high‑resolution timestamp** (in milliseconds with fractions).   
- Typical pattern to measure a block of code:
  - Take a timestamp **before** the code: `const t1 = performance.now();`  
  - Run the target code.  
  - Take a timestamp **after** the code: `const t2 = performance.now();`  
  - Compute duration: `const totalTime = t2 - t1;` and log or compare it. 

This gives an accurate numeric value of how long the code took to execute, which is then used to compare alternative approaches. 

***

## Example Setup: Creating 100 Paragraphs 

The scenario: display 100 `<p>` elements on the page, created dynamically via JavaScript. 

- Each paragraph is created using `document.createElement("p")`.   
- Text is assigned using `paragraph.textContent = "This is para " + i;` or similar.   
- The question: **How should these 100 elements be appended to the DOM for best performance?** 

Two baseline code versions are written and timed with `performance.now()`:

- **Code 1:** Append each paragraph directly to `document.body` inside the loop.   
- **Code 2:** Append each paragraph to a newly created `<div>`, then append the `<div>` to `document.body` once at the end. 

***

## Code 1: Direct Appends to `document.body` 

### Logic 

- Loop from 0 to 99.  
- For each iteration:
  - Create a `<p>` element.  
  - Set its `textContent`.  
  - Directly append it to `document.body` using `document.body.appendChild(p);`. 

### Measured performance 

- Using `performance.now()` before and after the loop, the total time is observed around **0.4–0.5 seconds** (value may vary per run and machine). 

### Hidden cost (many reflows & repaints) 

- Each `appendChild` call to `document.body`:
  - Triggers layout calculations (reflow).  
  - Triggers drawing on the screen (repaint).   
- With 100 paragraphs:
  - Roughly **100 reflows + 100 repaints** occur.   
- This repeated work makes the approach slower, especially as the number of elements grows. 

***

## Code 2: Append to `<div>`, Then Append `<div>` Once 

### Logic 

- Create a single `<div>` element: `const myDiv = document.createElement("div");`.   
- Loop from 0 to 99:
  - Create a `<p>` element.  
  - Set its `textContent`.  
  - Append the paragraph to `myDiv` using `myDiv.appendChild(p);`.   
- After the loop, append `myDiv` to `document.body` once. 

### Measured performance 

- Timed with `performance.now()` again, this pattern takes around **0.09 seconds**, significantly faster than Code 1. 

### Why is this faster? 

- Appending to `myDiv` does **not** immediately require updating the visible document until `myDiv` itself is attached to `document.body`.   
- Reflow and repaint happen **only when `myDiv` is appended to the actual document**, not for each paragraph insertion inside `myDiv`.   
- So, instead of 100 reflows + 100 repaints, there is effectively **1 reflow + 1 repaint** when `myDiv` is finally attached.   

This demonstrates that **minimizing DOM writes to the live document** can greatly improve performance. 

***

## Key Concepts: Reflow and Repaint 

### Viewport and pixels 

- The browser displays a page inside a **viewport/window** made up of many pixels.   
- To show elements, the browser:
  - Determines **where** each element should appear and its size.  
  - **Draws** those pixels with the correct colors and shapes. 

These steps map directly to **reflow** and **repaint**.

### What is Reflow? 

**Definition:**  
Reflow is the process of **calculating the dimensions and positions** of elements that need to be rendered on the page. 

Key points:

- Involves layout calculations: width, height, position (x, y), etc., for each affected element.   
- Is **computationally intensive** and often slower because it can involve recalculating layout for a tree of elements.   
- Requires more CPU time and browser resources.   
- Happens when operations modify layout‑affecting properties or structure of the DOM (e.g., adding/removing elements, changing size). 

Think of reflow as the browser **figuring out the geometry** of elements before drawing anything.

### What is Repaint? 

**Definition:**  
Repaint is the process of **drawing the visible pixels** of elements on the screen, pixel by pixel. 

Key points:

- Takes the already‑computed positions and sizes from reflow and **renders** them in the viewport.   
- Often **faster than reflow** because it does not recompute layout; it just draws.   
- Still consumes resources, but less than a full reflow in many cases. 

Think of repaint as **coloring in** elements after their layout is known.

### Relationship and performance impact 

- Reflow = layout computation → slower, heavy.  
- Repaint = drawing → faster than reflow, but still work.   
- More reflows + repaints → more time → slower UI.   
- Fewer reflows + repaints → faster, smoother UI. 

Therefore, a performance‑conscious developer aims to **structure DOM operations to minimize the number of reflows and repaints needed**. 

***

## Analyzing the Two Approaches Using Reflow & Repaint 

### Strategy 1 (Code 1: Direct to `body`) 

For each paragraph:

1. Create `<p>` and set text.  
2. Append `<p>` to `document.body`.  
3. Browser must:
   - Recalculate layout (reflow) to fit the new element.  
   - Draw the new state (repaint). 

With 100 paragraphs:

- About **100 reflows + 100 repaints**.   
- High cumulative cost, leading to ~0.5 seconds in the example. 

### Strategy 2 (Code 2: Use `<div>` as container) 

For each paragraph:

1. Create `<p>` and set text.  
2. Append `<p>` to `myDiv`.  
3. `myDiv` is not yet attached to `document.body`, so:
   - No immediate reflow and repaint for each paragraph. 

After the loop:

- Append `myDiv` to `document.body`.  
- Browser performs:
  - **One reflow** to integrate the `<div>` with all 100 paragraphs.  
  - **One repaint** to draw them. 

This shift from “100 small layout updates” to “1 batched layout update” is the core reason for the significant speedup. 

***

## Best Practice: Using `DocumentFragment` 

### What is `DocumentFragment`? 

- `DocumentFragment` is a **lightweight, in‑memory document‑like object**.   
- Behaves like a mini‑DOM tree that is **not directly attached to the main document**.   
- Consider it as a **temporary container** to build DOM structures off‑screen. 

Properties:

- Similar API to `document` for appending elements (e.g., `appendChild`).   
- Designed to demand fewer resources than manipulating the live document. 

### Key advantage for performance 

- **Appending elements to a `DocumentFragment` does not cause reflow or repaint.**   
- This means:
  - You can append 100, 1000, or even a million nodes to the fragment with **no layout recalculation** for each addition.   
- When the fragment itself is appended to the real document:
  - The browser performs **a single reflow + a single repaint** to integrate the entire batch.   

So the pattern is:

- In `DocumentFragment`:
  - **No reflow, no repaint** on each append.   
- When attaching fragment to `document`:
  - **One reflow + one repaint**. 

### Example pattern with `DocumentFragment` 

Typical code structure:

1. Create fragment:  
   - `const fragment = document.createDocumentFragment();`   
2. Loop to create 100 paragraphs:
   - Create `<p>`.  
   - Set `textContent`.  
   - Append to `fragment` with `fragment.appendChild(p);`.   
3. After the loop, append fragment to `document.body`:  
   - `document.body.appendChild(fragment);`. 

Characteristics:

- Lines where elements are appended to the fragment: **no reflow, no repaint**, regardless of how many children are added.   
- Final append of fragment to the actual document: **one reflow + one repaint**.   

This is often called the **“best code”** pattern in the lecture, because it maximizes batching and minimizes layout work. 

***

## Comparing Approaches 

| Approach                          | Where elements are appended during loop      | Reflows during creation | Repaints during creation | Final reflow/repaint when attached to document | Overall performance idea              |  
|-----------------------------------|----------------------------------------------|--------------------------|---------------------------|-----------------------------------------------|---------------------------------------|  
| Direct to `document.body` (Code 1) | Each `<p>` directly appended to `body`       | ~100                    | ~100                     | Already applied each time                      | Slowest among the three patterns  |  
| Use `<div>` container (Code 2)    | Each `<p>` appended to `<div>`, then `<div>` to `body` | 0 during inner appends   | 0 during inner appends    | 1 reflow + 1 repaint when `<div>` is attached | Much faster than Code 1         |  
| Use `DocumentFragment` (Best)     | Each `<p>` appended to `DocumentFragment`, then fragment to `body` | 0 during appends         | 0 during appends          | 1 reflow + 1 repaint when fragment is attached | Typically best, especially for large batches  |

Note: In real browsers, performance differences between `<div>` and `DocumentFragment` may vary by case and scale, but both are significantly better than repeatedly touching the live document inside tight loops. 

***

## Practical Guidelines for Writing Efficient DOM Code 

- **Batch DOM updates:**
  - Avoid adding or modifying DOM nodes one by one directly on `document.body` or other live parts of the DOM.   
  - Build structures in memory (via container nodes or `DocumentFragment`) and attach them in a single operation. 

- **Minimize reflows and repaints:**
  - Group style changes and DOM insertions to reduce how many times layout must be recomputed.   
  - Avoid unnecessary layout reads in between writes (not covered deeply here but conceptually related). 

- **Use `performance.now()` for real comparisons:**
  - When in doubt about which approach is faster, measure with `performance.now()` instead of guessing.   
  - Run tests multiple times to average out noise. 

- **Prefer `DocumentFragment` for larger or complex structures:**
  - Especially when adding many nodes or building nested structures, fragments give a clean and efficient way to batch DOM updates. 

***

## Summary of Main Takeaways 

- Performance optimization starts with **measuring**: `performance.now()` provides precise timing for code sections.   
- **Reflow** = layout calculation (positions and dimensions); it is **computationally heavy and slow**.   
- **Repaint** = pixel drawing on screen; **faster than reflow** but still has a cost.   
- Directly appending many elements to the live DOM triggers **many reflows and repaints**, making code slower.   
- Using an intermediate container (like a `<div>` or better, `DocumentFragment`) lets you:
  - Build the structure off‑screen.  
  - Trigger **only one reflow and one repaint** when you finally attach it to the actual document.   
- For efficient, scalable DOM manipulation:
  - **Minimize DOM writes to the live document**.  
  - **Batch changes** using containers or `DocumentFragment`.  
  - Always validate assumptions with timing measurements.